In [ ]:
import os
import subprocess
import tempfile
from pathlib import Path
import io

import nbformat

def _notebook_run(path):
    """Execute a notebook via nbconvert and collect output.
       :returns (parsed nb object, execution errors)
    """
    html_path = Path("html/" + path + ".html")
    real_path = Path(path)
    with tempfile.NamedTemporaryFile(dir=".", suffix=".ipynb", delete=False) as fout:
        name = fout.name
        fout.close()
        
        args = ["jupyter","nbconvert", "--to", "notebook", "--execute",
                "--ExecutePreprocessor.timeout=None", "--ExecutePreprocessor.allow_errors=True",
                "--output", fout.name, str(real_path)]
        
        try:
            subprocess.check_output(args, stderr=subprocess.STDOUT)
            
            ## convert to HTML as well
            args = ["jupyter","nbconvert", "--to", "html",
                "--ExecutePreprocessor.timeout=None",
                "--output", str(html_path), fout.name]
            subprocess.check_output(args, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            print(e)
            print(e.output)
            return
        fout = open(name, "r", encoding='utf-8')
        fout.seek(0)
        nb = nbformat.read(fout, nbformat.current_nbformat)
        fout.close()
        os.remove(name)

    errors = [[i+1,output] for i,cell in enumerate(nb.cells) if "outputs" in cell
                    for output in cell["outputs"]\
                            if output.output_type == "error"]

    return nb, errors

def _print_tracebacks(errors):
    if errors == []:
        print("Passed all tests!")
    for error in errors:
        print("Test failed in cell {}: {}: {}".format(error[0], error[1]['ename'], error[1]['evalue']))
        for line in error[1]['traceback']:
            print(line)
            
def _get_outputs(nb):
    return [[i,cell] for i,cell in enumerate(nb.cells) if "outputs" in cell]
    
def _print_stderr(nb):
    outputs = _get_outputs(nb)
    printed_output = [[cell[0], output] for cell in outputs for output in cell[1]['outputs'] if ('name' in output and output['name'] == 'stderr')]
    for out in printed_output:
        print("[{}]:\n{}".format(out[0], out[1]['text']))
        
def _print_stdout(nb):
    outputs = _get_outputs(nb)
    printed_output = [[cell[0], output] for cell in outputs for output in cell[1]['outputs'] if ('name' in output and output['name'] == 'stdout')]
    for out in printed_output:
        print("[{}]:\n{}".format(out[0], out[1]['text']))
        
def test_notebook(path, print_stdout=False, print_stderr=False):
    print("Testing: {}:...".format(path), end="")
    nb, errors = _notebook_run(path)
    if errors == []:
        print("PASSED")
    else:
        print("FAILED:")
        _print_tracebacks(errors)
    if print_stdout:
        _print_stdout(nb)
    if print_stderr:
        _print_stderr(nb)

In [ ]:
%%bash
mkdir -p html
cd ../../hardware/victims/firmware
rm -rf simpleserial-base-lab*
rm -rf glitch-simple-lab*
rm -rf simpleserial-aes-lab*

In [ ]:
test_notebook('PA_Intro_1-Firmware_Build_Setup.ipynb')
test_notebook('PA_Intro_2-Instruction_Differences.ipynb')

In [ ]:
test_notebook('PA_SPA_1-Timing_Analysis_with_Power_for_Password_Bypass.ipynb')

In [ ]:
#test_notebook('PA_DPA_1-Hamming_Weight_Measurement.ipynb') #FAILS, COMPILER MOVED SPOT
test_notebook('PA_DPA_2-Large_HW_Swings.ipynb')

In [ ]:
test_notebook('PA_CPA_1-Using_CW-Analyzer_for_CPA_Attack.ipynb')
test_notebook('PA_CPA_2-Manual_CPA_Attack.ipynb')
test_notebook('PA_CPA_3-Resynchronizing_Data_Traces.ipynb')
#test_notebook('PA_CPA_4-Hardware_Crypto_Attack.ipynb') #FAILS
test_notebook('PA_CPA_5-32bit_AES.ipynb') #MAY FAIL

In [ ]:
test_notebook('PA_Multi_1-Breaking_AES-256_Bootloader.ipynb')

In [ ]:
test_notebook('Fault_1-Introduction_to_Clock_Glitch_Attacks.ipynb')
test_notebook('Fault_3-Glitch_Buffer_Attacks.ipynb')